In [ ]:
import os
import face_recognition
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

def load_images_from_folder(folder):
    images = []
    labels = []
    for label_dir in os.listdir(folder):
        label_path = os.path.join(folder, label_dir)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                if os.path.isfile(img_path) and img_file.lower().endswith(('png', 'jpg', 'jpeg')):
                    img = face_recognition.load_image_file(img_path)
                    faces = face_recognition.face_encodings(img)
                    if len(faces) > 0:
                        images.append(faces[0])
                        labels.append(label_dir)
    return images, labels

def main():
    # Load dataset
    dataset_folder = 'Dataset'
    print("Loading dataset...")
    images, labels = load_images_from_folder(dataset_folder)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=42)

    # Train a SVM classifier
    print("Training classifier...")
    clf = svm.SVC(gamma='scale')
    clf.fit(X_train, y_train)

    # Evaluate the classifier
    print("Evaluating classifier...")
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # Ensure the models directory exists
    models_dir = 'models'
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    # Save the trained model
    model_path = os.path.join(models_dir, 'face_recognition_model.pkl')
    joblib.dump(clf, model_path)
    print(f"Model saved as {model_path}")

if __name__ == "__main__":
    main()


In [ ]:
import face_recognition
import joblib
import cv2


def recognize_face(image_path, model_path='models/face_recognition_model.pkl'):
    # Load the trained model
    clf = joblib.load(model_path)

    # Load the image
    image = face_recognition.load_image_file(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Find face locations and encodings in the image
    face_locations = face_recognition.face_locations(image_rgb)
    face_encodings = face_recognition.face_encodings(image_rgb, face_locations)

    for face_encoding, face_location in zip(face_encodings, face_locations):
        # Predict the label for the face encoding
        label = clf.predict([face_encoding])[0]

        # Draw a bounding box around the face
        top, right, bottom, left = face_location
        cv2.rectangle(image_rgb, (left, top), (right, bottom), (0, 255, 0), 2)

        # Display the label
        cv2.putText(image_rgb, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Display the result
    cv2.imshow(image_rgb)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    image_path = 'sample_images_for_test/download.jpeg'
    recognize_face(image_path)


In [ ]:
import os
import shutil

def delete_files_in_directory(directory_path):
  """Deletes all files in the specified directory.

  Args:
    directory_path: The path to the directory.
  """

  try:
    for filename in os.listdir(directory_path):
      file_path = os.path.join(directory_path, filename)
      if os.path.isfile(file_path):
        os.remove(file_path)
  except FileNotFoundError:
    print(f"Directory not found: {directory_path}")
  except OSError as e:
    print(f"Error deleting files: {e}")

# Example usage:
directory_to_clear = "/content/content/sample_data/Dataset/rahul"
delete_files_in_directory(directory_to_clear)


In [ ]:
!zip -r /content/my_folder.zip /content/sample_data/Dataset
